File Descriptions and Data Field Information
train.csv

    The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
    store_nbr identifies the store at which the products are sold.
    family identifies the type of product sold.
    sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
    onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

test.csv

    The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
    The dates in the test data are for the 15 days after the last date in the training data.

sample_submission.csv

    A sample submission file in the correct format.

stores.csv

    Store metadata, including city, state, type, and cluster.
    cluster is a grouping of similar stores.

oil.csv

    Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

holidays_events.csv

    Holidays and Events, with metadata
    NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
    Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

Additional Notes

    Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
    A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.


## importing libraries

In [1]:
#importing libraries
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### dataset

In [2]:
train_df = pd.read_csv("train.csv")
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


Categorical column

In [3]:
for col in train_df.select_dtypes(include=['object']).columns:
    print(col)

date
family


Numerical column

In [4]:
for col in train_df._get_numeric_data().columns:
    print(col)

id
store_nbr
sales
onpromotion


null checking

In [5]:
for col in train_df.columns:
    print(col, train_df[col].isnull().sum())

id 0
date 0
store_nbr 0
family 0
sales 0
onpromotion 0


In [17]:
oil_df = pd.read_csv("oil.csv")
oil_df = oil_df.fillna(method='ffill')
oil_df.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [7]:
holiday_df = pd.read_csv("holidays_events.csv")
holiday_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [8]:
stores_df = pd.read_csv("stores.csv")
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
transactions_df = pd.read_csv("transactions.csv")
transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [16]:
df_list = [oil_df, holiday_df, stores_df, transactions_df]
for i in df_list:
    for col in i.columns:
        print(col, i[col].isnull().sum())

date 0
dcoilwtico 1
date 0
type 0
locale 0
locale_name 0
description 0
transferred 0
store_nbr 0
city 0
state 0
type 0
cluster 0
date 0
store_nbr 0
transactions 0
